In [0]:
### UPDATE OUTBOUND  COUNT IN AUDIT TABLE
table_name = 'ebx_vm_vendor'

def update_s3_outbound_count(table_name,outbound_count):
    # Format the SQL query string
    query = f"""
    UPDATE dev.audit.audit_table
    SET s3_outbound_count = {outbound_count}
    WHERE table_name = '{table_name}'
    ;
    """ 
    spark.sql(query)

com.databricks.rpc.UnexpectedHttpException: Got invalid response: 504
	at com.databricks.rpc.Jetty9Client$$anon$1.onComplete(Jetty9Client.scala:991)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyComplete(ResponseNotifier.java:218)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyComplete(ResponseNotifier.java:210)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver.terminateResponse(HttpReceiver.java:481)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver.terminateResponse(HttpReceiver.java:461)
	at shaded.v9_4.org.eclipse.jetty.client.HttpReceiver.responseSuccess(HttpReceiver.java:424)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.messageComplete(HttpReceiverOverHTTP.java:374)
	at shaded.v9_4.org.eclipse.jetty.http.HttpParser.handleContentMessage(HttpParser.java:597)
	at shaded.v9_4.org.eclipse.jetty.http.HttpParser.parseContent(HttpParser.java:1722)
	at shaded.v9_4.org.eclipse.jetty.http.HttpParser.parseNext(HttpParser.java:1551)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.parse(HttpReceiverOverHTTP.java:208)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.process(HttpReceiverOverHTTP.java:148)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpReceiverOverHTTP.receive(HttpReceiverOverHTTP.java:80)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpChannelOverHTTP.receive(HttpChannelOverHTTP.java:131)
	at shaded.v9_4.org.eclipse.jetty.client.http.HttpConnectionOverHTTP.onFillable(HttpConnectionOverHTTP.java:172)
	at shaded.v9_4.org.eclipse.jetty.io.AbstractConnection$ReadCallback.succeeded(AbstractConnection.java:311)
	at shaded.v9_4.org.eclipse.jetty.io.FillInterest.fillable(FillInterest.java:105)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection$DecryptedEndPoint.onFillable(SslConnection.java:555)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection.onFillable(SslConnection.java:410)
	at shaded.v9_4.org.eclipse.jetty.io.ssl.SslConnection$2.succeeded(SslConnection.java:164)
	at shaded.v9_4.org.eclipse.jetty.io.FillInterest.fillable(FillInterest.java:105)
	at shaded.v9_4.org.eclipse.jetty.io.ChannelEndPoint$1.run(ChannelEndPoint.java:104)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.runTask(EatWhatYouKill.java:338)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:315)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:173)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:409)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.$anonfun$run$7(InstrumentedQueuedThreadPool.scala:158)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads(QueuedThreadPoolInstrumenter.scala:66)
	at com.databricks.instrumentation.QueuedThreadPoolInstrumenter.trackActiveThreads$(QueuedThreadPoolInstrumenter.scala:63)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool.trackActiveThreads(InstrumentedQueuedThreadPool.scala:124)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:153)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:883)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:1034)
	at java.base/java.lang.Thread.run(Thread.java:829)

In [0]:

from datetime import datetime
#import uuid

# table_name = "d"

def dump_extract(table_name,schema_name, folder_name):
   
    y = "s3a://unfi-data-dev-databricks-sap-extracts-s3/"
    #unique_id = str(uuid.uuid4())

    #define file content here
    file_content = spark.sql(f"SELECT * FROM dev.{schema_name}.{table_name}")
    
    
    # write it as a csv file with header in the destination/final_data.csv directory
    file_content.coalesce(1).write.mode("overwrite").format("csv").option("sep","|").option("header", "true").save(y + "/pre/" + table_name)
    
    
    
    for file in dbutils.fs.ls(y+"/pre/"+table_name):
        if not file.name.endswith("csv"):
            print(file.name)
            dbutils.fs.rm(file.path, True)
        # else :
        elif file.name.startswith("part-00000"):
            print(file.name)
            
            outbound_file_path= y + f"/{folder_name}"+ "/" + f"erp_gold_{table_name}" + ".csv"
            # Rename the file that starts with part-00000 in the destination/final_data.csv directory to data.csv, adds date followed by time to file name (testfile_ (replace this with actual file name))   
            dbutils.fs.mv(file.path, y + f"/{folder_name}"+ "/" + f"erp_gold_{table_name}" + ".csv")

    ### UPDATE OUTBOUND  COUNT IN AUDIT TABLE        
    if table_name =='vendor_v':
        table_name_in_audit ="ebx_vm_vendor"    
        df = spark.read.format("csv").option("delimiter", "|").load(outbound_file_path)
        outbound_count=df.count()
        update_s3_outbound_count(table_name_in_audit,outbound_count)



java.util.concurrent.TimeoutException: Futures timed out after [1 minute]
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:273)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:277)
	at scala.concurrent.Await$.$anonfun$result$1(package.scala:223)
	at scala.concurrent.BlockContext$DefaultBlockContext$.blockOn(BlockContext.scala:57)
	at scala.concurrent.Await$.result(package.scala:146)
	at com.databricks.backend.daemon.driver.DriverClient.executeWithoutPollingImpl(DriverClient.scala:355)
	at com.databricks.backend.daemon.driver.DriverClient.executeWithoutPolling(DriverClient.scala:416)
	at com.databricks.backend.daemon.driver.DriverClient.execute0(DriverClient.scala:556)
	at com.databricks.backend.daemon.driver.DriverClient.execute(DriverClient.scala:313)
	at com.databricks.webapp.notebook.command.CommandRunner.execute(CommandRunner.scala:404)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.$anonfun$runCommandPlan$2(NotebookManager.scala:5220)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.withAttributionContext(NotebookManager.scala:1536)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.runCommandPlan(NotebookManager.scala:5181)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager._internalRunCommand(NotebookManager.scala:5050)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.$anonfun$runCommand$1(NotebookManager.scala:4874)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:668)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:686)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.common.util.locks.LoggedLock$.withAttributionContext(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:472)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at com.databricks.common.util.locks.LoggedLock$.withAttributionTags(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:663)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:582)
	at com.databricks.common.util.locks.LoggedLock$.recordOperationWithResultTags(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:542)
	at com.databricks.common.util.locks.LoggedLock$.recordOperation(LoggedLock.scala:89)
	at com.databricks.common.util.locks.LoggedLock$.withLock(LoggedLock.scala:160)
	at com.databricks.common.util.locks.LoggedReadWriteLock.withWriteLock(LoggedReadWriteLock.scala:36)
	at com.databricks.webapp.notebook.NotebookManager.com$databricks$webapp$notebook$NotebookManager$$withLockOrTimeout(NotebookManager.scala:309)
	at com.databricks.webapp.n

In [0]:

gold_schema ='gold_erp'
view_list =["bus_party_addr_v","bus_party_indvdl_v","bus_party_org_v","vendor_v","ref_vendor_type_v","vendor_bus_sys_relnshp_v","vendor_pmt_data_pmt_methods_v","vendor_pmt_data_pmt_terms_v"]
for table_name in view_list :
    dump_extract(table_name,gold_schema,'Vendor')


java.util.concurrent.TimeoutException: Futures timed out after [1 minute]
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:273)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:277)
	at scala.concurrent.Await$.$anonfun$result$1(package.scala:223)
	at scala.concurrent.BlockContext$DefaultBlockContext$.blockOn(BlockContext.scala:57)
	at scala.concurrent.Await$.result(package.scala:146)
	at com.databricks.backend.daemon.driver.DriverClient.executeWithoutPollingImpl(DriverClient.scala:355)
	at com.databricks.backend.daemon.driver.DriverClient.executeWithoutPolling(DriverClient.scala:416)
	at com.databricks.backend.daemon.driver.DriverClient.execute0(DriverClient.scala:556)
	at com.databricks.backend.daemon.driver.DriverClient.execute(DriverClient.scala:313)
	at com.databricks.webapp.notebook.command.CommandRunner.execute(CommandRunner.scala:404)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.$anonfun$runCommandPlan$2(NotebookManager.scala:5220)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.withAttributionContext(NotebookManager.scala:1536)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.runCommandPlan(NotebookManager.scala:5181)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager._internalRunCommand(NotebookManager.scala:5050)
	at com.databricks.webapp.notebook.NotebookManager$InternalUserSpecificManager.$anonfun$runCommand$1(NotebookManager.scala:4874)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:668)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:686)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:418)
	at com.databricks.common.util.locks.LoggedLock$.withAttributionContext(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:472)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:455)
	at com.databricks.common.util.locks.LoggedLock$.withAttributionTags(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:663)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:582)
	at com.databricks.common.util.locks.LoggedLock$.recordOperationWithResultTags(LoggedLock.scala:89)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:542)
	at com.databricks.common.util.locks.LoggedLock$.recordOperation(LoggedLock.scala:89)
	at com.databricks.common.util.locks.LoggedLock$.withLock(LoggedLock.scala:160)
	at com.databricks.common.util.locks.LoggedReadWriteLock.withWriteLock(LoggedReadWriteLock.scala:36)
	at com.databricks.webapp.notebook.NotebookManager.com$databricks$webapp$notebook$NotebookManager$$withLockOrTimeout(NotebookManager.scala:309)
	at com.databricks.webapp.n

In [0]:
# silver_schema ='silver_erp'
# view_list =["ebx_bp_org_address","ebx_bp_individ_address"]
# for table_name in view_list :
#     dump_extract(table_name,silver_schema,'Vendor')


In [0]:
#  dbutils.fs.ls("s3a://unfi-data-dev-databricks-sap-extracts-s3/Vendor/")

True